**Task 07: Querying RDF(s)**

In [2]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 7.6MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 


Read the RDF file

In [3]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=Ndc3ba89e33a04c1aa7bd4026c28ec793 (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

With **RDFLib**:

First we list the subclasses directly under the class "Person" and then we look for subclasses of these subclasses:

In [4]:
ns = Namespace("http://somewhere#")
for s, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s)
  for s2, p, o in g.triples((None, RDFS.subClassOf, s)):
    print(s2)

http://somewhere#Researcher
http://somewhere#PhDstudent


"Researcher" and "PhDstudent" are subclasses of the class "Person".


---
With **SPARQL**


We import the "prepareQuery" plug in and use SPARQL to list the subclasses of the class "Person". We use "rdfs:subClassOf/rdfs:subClassOf*" to include possible subclasses of these subclasses.  

In [10]:
from rdflib.plugins.sparql import prepareQuery

q1 = prepareQuery('''
  SELECT DISTINCT ?x 
  WHERE 
  {
	?x rdfs:subClassOf/rdfs:subClassOf* ns:Person
  }
  ''',
  initNs = { "ns": ns}
)
for x in g.query(q1):
  print(x)

(rdflib.term.URIRef('http://somewhere#Researcher'),)
(rdflib.term.URIRef('http://somewhere#PhDstudent'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


With **RDFLib**:

We first list all the individuals of "Person" and then the individuals of its subclasses ("Researcher" and "PhDstudent"):

In [5]:
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  print(s)
for sc, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s, p, o in g.triples((None, RDF.type, sc)):
    print(s)

http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith


---
With **SPARQL**:

We create two queries: "q1" in order to find all the individuals of "Person" and "q2" to find individuals of the subclasses of "Person".

In [ ]:

q1 = prepareQuery('''
  SELECT DISTINCT ?x
  WHERE 
  {
	?x a ns:Person
  }
  ''',
  initNs = { "ns": ns}
)
for r in g.query(q1):
  print(r)
  
q2 = prepareQuery('''
  SELECT DISTINCT ?y 
  WHERE 
  {
	?x rdfs:subClassOf/rdfs:subClassOf* ns:Person .
  ?y a ?x
  }
  ''',
  initNs = { "ns": ns}
)
for r in g.query(q2):
  print(r)

(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


With **RDFLib**:

We first list the properties of all the individuals of "Person" and then the properties of the individuals of its subclasses ("Researcher" and "PhDstudent"):

In [8]:
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  for s2, p2, o2 in g.triples((s, None, None)):
    print(s2, p2, o2)

for sc, p, o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s, p, o in g.triples((None, RDF.type, sc)):
    for sind, pind, oind in g.triples((s, None, None)):
      print(sind, pind, oind)

http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http://somewhere#http://www

---
With **SPARQL**:

We find all the individuals of "Person" and its subclasses and then list the properties of each of them:

In [11]:
q1 = prepareQuery('''
  SELECT ?x ?y ?z
  WHERE{
  ?x rdf:type/rdfs:subClassOf* ns:Person.
  ?x ?y ?z
  }
  ''',
  initNs = { "ns": ns}
)
for r in g.query(q1):
  print(r)

(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given'), rdflib.term.Literal('John', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN'), rdflib.term.Literal('John Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://somewhere#Person'))
(rdflib.term.URIRef('http://somewhere#JohnSmith'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family'), rdflib.term.Literal('Smith', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#http://www.w3.org/2001/vcard-rdf/